# Использование API в открытом доступе. Hotellook
Ссылка на сайт: https://search.hotellook.com/

In [2]:
import requests
import json
import pandas as pd

Переходим на страницу [с отелями в Москве](https://search.hotellook.com/hotels?=1&cityId=12153), указываем любые параметры заселения. Изначально мы планировали получить информацию из сайта методом парсинга, однако после изучения исходного кода главной страницы и отдельной страницы для конкретного отеля было выявлено, что даже если на если выгруженных html можно получить информацию, то она не будет особо полезной, а затраты на парсинг плохо структурированных страниц будут крайне велики.

После данного вывода, было решено рассмотреть внешние запросы, которые сайт делает во время загрузки. С помощью панели "Сеть" в инструментах разработчика браузера Microsoft Edge был найден интересный запрос на адрес https://search.hotellook.com/api/wl_search/result Название сайта сразу же привлекло наше внимание, поэтому мы решили скопировать запрос в виде curl (пример запроса смотри ниже), чтобы проверить, поможет ли данная находка в сборе информации. 

```bash
curl 'https://search.hotellook.com/api/wl_search/result' \
  -H 'accept: */*' \
  -H 'accept-language: ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7' \
  -H 'content-type: application/json' \
  -b '_ym_uid=1740687636891539769; _ym_d=1740687636; _ym_isad=2; _fbp=fb.1.1740687635990.94303087349110335; _ym_visorc=w; locale=en; currency=RUB; _sp_ses.b93a=*; SERVERID=s1; hide_hotels=true; marker=direct.Zzf9414da07bac412a8081a93-126017; hotel_search_params=%7B%22params_attributes%22%3A%7B%22check_in%22%3A%222025-03-06%22%2C%22check_out%22%3A%222025-03-13%22%2C%22destination%22%3A%7B%22name%22%3A%22Moscow%22%2C%22location_id%22%3A12153%7D%7D%7D; __gads=ID=44dcaed736b18c79:T=1740690590:RT=1740690590:S=ALNI_MZXwNbPSlTOoNneAx98XOLGWMR8Tg; __gpi=UID=00001047f9069e5d:T=1740690590:RT=1740690590:S=ALNI_MavMYFmwoZLswhjRNWA5OEp3F6L2w; __eoi=ID=b27dad9016678eb9:T=1740690590:RT=1740690590:S=AA-AfjZC8WTywdSjZyrYKFv1Ea1G; _sp_id.b93a=0645083f-b576-46c5-ad0e-2bab35cc307c.1740687637.1.1740690593.1740687637.0c51d766-a433-4071-8ee4-45b25167c7f0' \
  -H 'origin: https://search.hotellook.com' \
  -H 'priority: u=1, i' \
  -H 'referer: https://search.hotellook.com/hotels?=1&adults=2&checkIn=2025-03-06&checkOut=2025-03-13&children=&cityId=12153&currency=rub&destination=Moscow&language=en&marker=direct.Zzf9414da07bac412a8081a93-126017' \
  -H 'sec-ch-ua: "Not(A:Brand";v="99", "Microsoft Edge";v="133", "Chromium";v="133"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Windows"' \
  -H 'sec-fetch-dest: empty' \
  -H 'sec-fetch-mode: cors' \
  -H 'sec-fetch-site: same-origin' \
  -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0' \
  --data-raw '{"page":"serp","search_id":"23882775-d9cb-466e-8077-68984e8b2c3f","params":{"check_in":"2025-03-06","check_out":"2025-03-13","marker":"direct.Zzf9414da07bac412a8081a93-126017","currency":"rub","locale":"en","rooms":[{"adults":2,"children":[]}],"locations_ids":[12153],"hotels_ids":[],"destination":"Moscow","host":"search.hotellook.com","flags":{"auid":null,"ab":null,"deviceType":"mobile"},"popularity":"default"},"selected_hotels_ids":[],"filters":{"prices":{"groups":[0,500,1000,1500,2000,2500,3000,4000,5000,6000,7000,8000,9000,12000,15000,20000,30000,50000,100000,2147483647]},"ratings":{"groups":[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]}},"sort":"popularity","limit":10,"offset":0}'
```

Помимо самого запроса и сопровождающих заголовков можно заметить "сырые данные" data-raw, это параметры запроса к API hotellook. Параметры указаны в виде JSON-данных. Мы можем воссоздать такой же запрос с помощью питона, используя библиотеку requests. Заодно уберем лишние заголовки и параметры, без которых ответ запрос не изменится

In [8]:
def get_all_hotellook_data(limit, location_id=[12153]):
    url = "https://search.hotellook.com/api/wl_search/result"
    
    headers = {
        "accept": "*/*",
        "accept-language": "ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
        "content-type": "application/json",
        "origin": "https://search.hotellook.com",
        "priority": "u=1, i",
        "referer": f"https://search.hotellook.com/hotels?=1&cityId={location_id}",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0"
    }
    
    payload = {
        "page": "serp",
        "search_id": "",
        "params": {
            "check_in": "2025-02-28",
            "check_out": "2025-03-01",
            "currency": "rub",
            "locale": "ru",
            "rooms": [{"adults": 2, "children": []}],
            "locations_ids": location_id,
            "destination": None,
            "hotels_ids": [],
            "host": "search.hotellook.com",
            "flags": {"auid": None, "ab": None, "deviceType": "mobile"},
            "popularity": "default"
        },
        "sort": "popularity",
        "limit": limit,
        "offset": 0
    }
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        data = response.json()
        with open(f"hotellook_alldata_limit-{limit}.json", "w", encoding="utf-8") as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
            print(f"Данные загружены в hotellook.json. Число запросов: {limit}")
    else:
        print(f"Ошибка {response.status_code}: {response.text}")
    return 


In [9]:
get_all_hotellook_data(10)

Данные загружены в hotellook.json. Число запросов: 10


В данном response есть полезные данные, которые затем будут использованы в проекте. Напишем функцию, которая будет обрабатывать полученный json и отбирать всю возможную информацию о каждом отеле.

In [10]:
def extract_full_hotel_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    hotels = data.get("hotels", [])

    full_hotel_data = []
    for hotel in hotels:
        hotel_info = {
            "id": hotel.get("id"),
            "name": hotel.get("name"),
            "address": hotel.get("address"),
            "location_id": hotel.get("location_id"),
            "latitude": hotel.get("location", {}).get("lat"),
            "longitude": hotel.get("location", {}).get("lon"),
            "stars": hotel.get("stars"),
            "rating": hotel.get("rating"),
            "reviews_count": hotel.get("reviews_count"),
            "min_price": hotel.get("minprice"),
            "median_min_price": hotel.get("median_minprice"),
            "distance_to_center": hotel.get("distance_to_center"),
            "chain": hotel.get("chain"),
            "property_type": hotel.get("property_type"),
            "photos_count": hotel.get("photos_count"),
            "amenities": hotel.get("amenities"),
            "badges": [badge["code"] for badge in (hotel.get("badges") or {}).get("badges", [])],
            "check_in": hotel.get("check_in"),
            "check_out": hotel.get("check_out"),
            "year_opened": hotel.get("year_opened"),
            "year_renovated": hotel.get("year_renovated"),
            "popularity": hotel.get("popularity"),
            "popularity2": hotel.get("popularity2"),
            "trending_speed": hotel.get("trending_speed"),
            "photos_ids": hotel.get("photos_ids"),
            "pois_distances": hotel.get("pois_distances"),
            "nearest_poi_by_category": hotel.get("nearest_poi_by_category"),
            "districts_ids": hotel.get("districts_ids"),
            "weights": hotel.get("weights"),
            "scoring": hotel.get("scoring"),
            "debug_info": hotel.get("debug_info"),
        }
        full_hotel_data.append(hotel_info)

    return pd.DataFrame(full_hotel_data)

In [11]:
file_path = "hotellook_alldata_limit-10.json"
df_full_hotels = extract_full_hotel_data(file_path)
df_full_hotels

,id,name,address,location_id,latitude,longitude,stars,rating,reviews_count,min_price,...,popularity,popularity2,trending_speed,photos_ids,pois_distances,nearest_poi_by_category,districts_ids,weights,scoring,debug_info
0,333570,Отель Вега Измайлово,Измайловское шоссе 71 строение 3В,12153,55.790752,37.747135,4,9.0,15712,46,...,16804,2268,0.00,"[7489913361, 8352837505, 8352837517, 748991281...","{'69873': 35331, '69874': 42911, '69885': 2839...","{'airport': 69885, 'metro_station': 72010, 'tr...",[1416636],{'default': 2.2766292},{},
1,1198039,Sunflower Авеню Отель Москва,"Ул. Щепкина 32, стр.1",12153,55.779600,37.629820,3,8.9,3195,35,...,16803,1399,0.89,"[8693498484, 8693498482, 8693498477, 869349848...","{'69873': 28861, '69874': 43984, '69885': 2465...","{'airport': 69885, 'metro_station': 71913, 'tr...","[1667834, 1676723]",{'default': 2.2280583},{},
2,333559,Холидей Инн Москва Сокольники,Русаковская ул.24,12153,55.787690,37.680330,4,9.0,9624,53,...,16802,1855,0.18,"[7972531316, 7949733813, 7862423210, 791411950...","{'69873': 31827, '69874': 43714, '69885': 2579...","{'airport': 69885, 'metro_station': 72006, 'tr...",[1416636],{'default': 2.2625914},{},
3,1704466136,Хилтон Гарден Инн Москва Красносельская,"Улица Верхняя Красносельская, 11а-4",12153,55.784810,37.662830,4,9.1,4499,55,...,16801,486,0.69,"[8725235261, 8725235254, 8725235256, 872523524...","{'69873': 30787, '69874': 43771, '69885': 2536...","{'airport': 69885, 'metro_station': 71908, 'tr...","[1667837, 1676723]",{'default': 2.2162018},{},
4,333572,Отель Бородино,Русаковская ул.13/5,12153,55.784580,37.672400,4,8.9,4871,47,...,16800,918,0.39,"[7761510695, 7504780382, 7760836716, 750478632...","{'69873': 31225, '69874': 43546, '69885': 2575...","{'airport': 69885, 'metro_station': 71908, 'tr...","[1667837, 1676723]",{'default': 2.212328},{},
5,44463003,ibis Москва Динамо,"Ленинградский Проспект 37, корпус 8",12153,55.792400,37.543470,3,8.7,6839,39,...,16799,1215,0.25,"[7630677409, 8793151888, 8863392105, 886339209...","{'69873': 26286, '69874': 47594, '69885': 2089...","{'airport': 69885, 'beach': 323310, 'metro_sta...",[1467379],{'default': 2.172786},{},
6,1899369125,Арт Москва Войковская,Ulitsa Kosmonavta Volkova 6А,12153,55.815300,37.512740,4,8.8,2343,42,...,16798,728,2.78,"[9525299954, 9525299955, 9525299950, 952529994...","{'69873': 27341, '69874': 50748, '69885': 1783...","{'airport': 69885, 'beach': 323310, 'metro_sta...",[1467379],{'default': 2.1888375},{},
7,276236,Гостиница Альфа Измайлово,Измайловское шоссе 71 корпус А,12153,55.789787,37.749550,4,8.6,18605,38,...,16797,2567,0.00,"[7487223299, 7761608657, 7761609184, 776160902...","{'69873': 35390, '69874': 42772, '69885': 2857...","{'airport': 69885, 'metro_station': 72010, 'tr...",[1416636],{'default': 2.195934},{},
8,276243,Москоу Холидэй Отель,"Улица Мневники, 3, стр. 2",12153,55.773730,37.499120,4,8.3,3282,33,...,16796,413,1.59,"[7499262649, 7499265095, 7925002564, 749926063...","{'69873': 22981, '69874': 47199, '69885': 2202...","{'airport': 69885, 'metro_station': 71872}",[1667879],{'default': 2.1297414},{},
9,1898005746,Покровка 6 Отель,"улица Покровка, д. 6, Москва",12153,55.758730,37.642040,3,9.6,0,38,...,16795,326,0.00,"[8734876847, 8734876854, 8734876866, 873487685...","{'69873': 27938, '69874': 41548, '69885': 2703...","{'airport': 69885, 'metro_station': 71875, 'tr...","[1667836, 1676723]",{'default': 2.1378424},{},


У нас получилось загрузить в датасет первые данные в их изначальном формате - отбором полезных столбцов и преобразованием займемся позже на этапе EDA. Сейчас нам необходимо выгрузить все возможные данные

Перед этим убедимся, что запрос ко всем отелям в городе и запрос к конкретному отелю не различается в объеме предоставляемой информации. Создадим функцию, которая делает запрос только к одному отеля, повторив все ранее описанные действия - зайдем на страницу отеля, подсмотрим и честно воспользуемся найденным запросом к API, перепишем все на питон и сохраним результат. 

In [20]:
def get_single_hotel_data(hotel_id):
    url = "https://search.hotellook.com/api/wl_search/result"
    
    headers = {
        "accept": "*/*",
        "accept-language": "ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
        "content-type": "application/json",
        "origin": "https://search.hotellook.com",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0"
    }
    
    payload = {
        "page": "hotel_page",
        "search_id": "7c33f3c5-37ab-4ead-88c4-84c31e121e33",
        "params": {
            "marker": "direct.Zz1f6b911e8a60403ca251d64-126017",
            "currency": "rub",
            "locale": "ru",
            "locations_ids": [],
            "hotels_ids": [hotel_id],
            "host": "search.hotellook.com",
            "flags": {"auid": None, "ab": None, "deviceType": "desktop"},
            "popularity": "default"
        },
        "selected_hotels_ids": [],
        "filters": {},
        "sort": "popularity",
        "limit": 1,
        "offset": 0
    }
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        data = response.json()
        with open(f"hotellook_single_{hotel_id}.json", "w", encoding="utf-8") as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        return data
    else:
        print(f"Ошибка {response.status_code}: {response.text}")
        return None

get_single_hotel_data(1198039)


{'search_id': '',
 'stop': True,
 'locations': {'12196': {'seasons': {},
   'name': 'Санкт-Петербург',
   'country_name': 'Россия',
   'country_id': 186,
   'declensions': {'1': 'Санкт-Петербург',
    '2': 'Санкт-Петербурга',
    '3': 'Санкт-Петербургу',
    '4': 'Санкт-Петербург',
    '5': 'Санкт-Петербургом',
    '6': 'Санкт-Петербургe',
    '7': 'в Санкт-Петербурге'},
   'property_types_count': {'by_types': {'0': 58,
     '1': 729,
     '2': 40,
     '3': 11,
     '4': 2656,
     '5': 2,
     '6': 113,
     '7': 94,
     '8': 6},
    'total': 3709},
   'timezone': 'Europe/Moscow',
   'iatas': [],
   'center_coords': {'lat': 59.95, 'lon': 30.316668},
   'full_name': 'Санкт-Петербург, Россия',
   'pois_ids': [],
   'latin_name': 'St. Petersburg',
   'latin_country_name': 'Russia',
   'id': 12196,
   'zones': ['eurasia', 'asia', 'europe'],
   'state_code': '',
   'code': 'LED',
   'country_code': 'RU',
   'latin_full_name': 'St. Petersburg, Russia',
   'path': '186.1416746.12196',
   '

In [13]:
df_single_hotel = extract_full_hotel_data("hotellook_single_1198039.json")
df_single_hotel

,id,name,address,location_id,latitude,longitude,stars,rating,reviews_count,min_price,...,popularity,popularity2,trending_speed,photos_ids,pois_distances,nearest_poi_by_category,districts_ids,weights,scoring,debug_info
0,1198039,Sunflower Авеню Отель Москва,"Ул. Щепкина 32, стр.1",12153,55.7796,37.62982,3,8.9,3195,35,...,610449,1399,0,"[8693498484, 8693498482, 8693498477, 869349848...","{'69873': 28861, '69874': 43984, '69885': 2465...","{'airport': 69885, 'metro_station': 71913, 'tr...","[1667834, 1676723]",{},{},


Как мы и предполагали, данные во время общего запроса и в виде отдельного запроса ничем не отличаются - те же столбцы и те же данные. Это значительно упрощает нашу работу

Начнем полноценно выкачивать данные из API, настроив limit и указав неколько городов. До этого по умолчанию у нас стоял город с id 12153, это Москва. Но на сайте hotellooks есть и другие локации: Санкт-Петербург, Сочи, Анапа. В этих городах сравнительно большое количество отелей. Отели других в городах России в базе данных hotellook немногочисленны  (Владивосток - 252 отеля, Владикавказ - 91 отель, Владимир - 63 отеля, Тула - 67 отелей, Нижний Новгород - 242 отеля, Ростов-на-Дону - 253 отеля). 

Поскольку городов с большим числом (>1000 отелей) отелей немного, позволим вручную найти id данных городов


In [14]:
# 12153 - Москва, всего 9 163 значений
# 12196 - Санкт-Петербург, 13 173 значений
# 12193 - Сочи, 14 220 значений
# 12099 - Анапа, 5 565 значений
 
get_all_hotellook_data(limit=50000, location_id=[12153, 12196, 12193, 12099])

Данные загружены в hotellook.json. Число запросов: 50000


In [38]:
file_path = "hotellook_alldata_limit-50000.json"
df_full_hotels = extract_full_hotel_data(file_path)
df_full_hotels

,id,name,address,location_id,latitude,longitude,stars,rating,reviews_count,min_price,...,popularity,popularity2,trending_speed,photos_ids,pois_distances,nearest_poi_by_category,districts_ids,weights,scoring,debug_info
0,1250022545,Мини-Отель Мопс на Рубинштейна,"улица Рубинштейна, д.12, Санкт-Петербург",12196,59.931152,30.345670,3,9.2,498,19,...,69104,291,1.6,"[7515131692, 8286984314, 7515134055, 751513409...","{'71006': 14062, '71781': 732, '71817': 433, '...","{'airport': 71006, 'beach': 346587, 'metro_sta...",[1664298],{'default': 2.2985132},{},
1,964250854,Мини Отель Mushroom,"Набережная Канала Грибоедова,42",12196,59.930200,30.319010,3,9.3,0,17,...,69103,232,0.0,"[7510410856, 9611111725, 9400656506, 940065650...","{'71006': 13814, '71842': 422, '71862': 405, '...","{'airport': 71006, 'beach': 346587, 'metro_sta...",[1416598],{'default': 2.0068567},{},
2,333570,Отель Вега Измайлово,Измайловское шоссе 71 строение 3В,12153,55.790752,37.747135,4,9.0,15712,46,...,69102,2268,0.0,"[7489913361, 8352837505, 8352837517, 748991281...","{'69873': 35331, '69874': 42911, '69885': 2839...","{'airport': 69885, 'metro_station': 72010, 'tr...",[1416636],{'default': 2.2766292},{},
3,21120269,Версаль Отель,ул.Восстания д.12,12196,59.936010,30.360070,3,9.2,415,27,...,69101,107,0.0,"[7743613705, 7743613701, 7743613699, 651834590...","{'71006': 14733, '71817': 1158, '71843': 577, ...","{'airport': 71006, 'beach': 346587, 'metro_sta...",[1664298],{'default': 2.1939864},{},
4,1910372927,Отель Реноме,Гончарная улица 26,12196,59.928090,30.369590,4,9.1,0,23,...,69099,566,0.0,"[9011295402, 9011295366, 9011295384, 901129539...","{'71006': 13991, '71753': 1003, '71843': 884, ...","{'airport': 71006, 'beach': 346587, 'metro_sta...",[1664298],{'default': 2.1731336},{},
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42117,1915703569,"2-комн. квартира, 40 кв.м. на 4 человека","Воскресенская, 14/1",12193,43.402145,39.979725,0,0.0,0,193,...,121,0,0.0,"[9686324383, 9686324384, 9686324385, 968632438...","{'71130': 6056, '302866': 974, '302867': 1978,...","{'airport': 71130, 'beach': 302866}","[12096, 1672960, 1676737]",{'default': 0.17178896},{},
42118,1915947571,"1-комн. квартира, 25 кв.м. на 3 человека","Воскресенская улица, 14/1",12193,43.402195,39.979725,0,0.0,0,185,...,120,0,0.0,"[9702267731, 9702267732, 9702267733, 970226773...","{'71130': 6052, '302866': 978, '302867': 1983,...","{'airport': 71130, 'beach': 302866}","[12096, 1672960, 1676737]",{'default': 0.17178896},{},
42119,1915663041,"1-комн. квартира, 25 кв.м. на 3 человека","Воскресенская улица, 14",12193,43.398132,39.980553,0,0.0,0,173,...,118,0,0.0,"[9665066098, 9665066099, 9665066100, 9665066101]","{'71130': 6475, '302866': 681, '302867': 1554,...","{'airport': 71130, 'beach': 302866}","[12096, 1672960, 1676737]",{'default': 0.17178896},{},
42120,1915723179,3-комнатная квартира,"Нижнеимеретинская улица, 137А",12193,43.397510,39.968616,0,0.0,0,109,...,113,0,0.0,"[9686705995, 9719167368, 9719167369, 971916737...","{'71130': 6124, '302866': 59, '302867': 2169, ...","{'airport': 71130, 'beach': 302866}","[12096, 1672960, 1676737]",{'default': 0.17178896},{},


In [37]:
df_full_hotels.to_csv("hotellook.csv", index=False)